# Tarefa Numérica 2

Algoritmo no livro:

1. Dá um chute do valor do passo
2. Estima o erro com três $\kappa_i$
3. Reduz o passo até que o erro esteja dentro da tolerância
4. Avança a solução com uma fórmula precisa
5. Repete até o tempo final


In [20]:
import numpy as np
import pandas as pd

# Definição do problema de Cauchy com solução exata conhecida
# y' = -2 t y, y(0) = 1 → solução exata: y(t) = exp(-t^2)
def f(t, y):
    return -2 * t * y

def y_exact(t):
    return np.exp(-t**2)

# Implementação de métodos Runge-Kutta

# RK3 clássico (ordem 3)
def rk3_step(t, y, h):
    k1 = f(t, y)
    k2 = f(t + h/2, y + (h/2) * k1)
    k3 = f(t + h, y - h * k1 + 2 * h * k2)
    return y + (h/6) * (k1 + 4 * k2 + k3)

# RK4 clássico (ordem 4)
def rk4_step(t, y, h):
    k1 = f(t, y)
    k2 = f(t + h/2, y + (h/2) * k1)
    k3 = f(t + h/2, y + (h/2) * k2)
    k4 = f(t + h, y + h * k3)
    return y + (h/6) * (k1 + 2 * k2 + 2 * k3 + k4)

# Função para gerar tabela de convergência de um método dado
def convergence_table(method_step, t_final=0.5, n_values=[4, 8, 16, 32, 64, 128]):
    results = []
    for n in n_values:
        h = t_final / n
        # Calcula solução com passo h
        y_h = 1.0  # condição inicial
        t = 0.0
        for _ in range(n):
            y_h = method_step(t, y_h, h)
            t += h
        
        # Estima y com n=1000
        y_aprox = 1.0
        t = 0.0
        h_aprox = t_final/1000
        for _ in range(1000):
            y_aprox = method_step(t, y_aprox, h_aprox)
            t += h_aprox
        
        # Estimativa de erro usando y aproximado
        e_est = abs(y_h - y_aprox)
        
        # Erro exato
        e_exact = abs(y_h - y_exact(t_final))
        
        results.append((n, h, e_exact, e_est))
    
    # Construir tabela com estimativa de ordem
    table = []
    for i, (n, h, e_exact, e_est) in enumerate(results):
        if i == 0:
            table.append((n, h, e_exact, e_est, np.nan, np.nan, np.nan, np.nan))
        else:
            # Para erro exato
            _, _, e_exact_prev, _ = results[i-1]
            q_exact = abs(e_exact_prev / e_exact)
            p_exact = np.log2(q_exact)
            
            # Para erro estimado
            _, _, _, e_est_prev = results[i-1]
            q_est = abs(e_est_prev / e_est)
            p_est = np.log2(q_est)
            
            table.append((n, h, e_exact, q_exact, p_exact, e_est, q_est, p_est))
    
    df = pd.DataFrame(table, columns=[
        "n", "h", "|e(t,h)| (exato)", "q (exato)", "p* (exato)", 
        "|e(t,h)| (estimado)", "q (estimado)", "p* (estimado)"
    ])
    return df

# Gerar e exibir tabelas para RK3 e RK4
df_rk3 = convergence_table(rk3_step)
df_rk4 = convergence_table(rk4_step)

print("Tabela para RK3:")
display(df_rk3)
print("\nTabela para RK4:")
display(df_rk4)

Tabela para RK3:


,n,h,"|e(t,h)| (exato)",q (exato),p* (exato),"|e(t,h)| (estimado)",q (estimado),p* (estimado)
0,4,0.125000,1.371068e-04,0.000137,NaN,NaN,NaN,NaN
1,8,0.062500,1.605161e-05,8.541623,3.094510,1.605161e-05,8.541627,3.094511
2,16,0.031250,1.942681e-06,8.262607,3.046597,1.942674e-06,8.262636,3.046602
3,32,0.015625,2.389747e-07,8.129234,3.023119,2.389670e-07,8.129464,3.023160
4,64,0.007812,2.963438e-08,8.064104,3.011514,2.962667e-08,8.065942,3.011843
5,128,0.003906,3.689574e-09,8.031925,3.005746,3.681864e-09,8.046650,3.008388



Tabela para RK4:


,n,h,"|e(t,h)| (exato)",q (exato),p* (exato),"|e(t,h)| (estimado)",q (estimado),p* (estimado)
0,4,0.125000,4.523219e-08,4.523219e-08,NaN,NaN,NaN,NaN
1,8,0.062500,3.075462e-09,1.470744e+01,3.878474,3.075460e-09,14.707454,3.878476
2,16,0.031250,3.599920e-10,8.543140e+00,3.094766,3.599898e-10,8.543187,3.094774
3,32,0.015625,2.746348e-11,1.310803e+01,3.712379,2.746126e-11,13.109008,3.712487
4,64,0.007812,1.867284e-12,1.470771e+01,3.878501,1.865064e-12,14.724031,3.880101
5,128,0.003906,1.215694e-13,1.535982e+01,3.941089,1.193490e-13,15.626977,3.965967
